# NVIDIA Nemotron - Exemplos de Uso

O **Nemotron** é uma família de modelos da NVIDIA com arquitetura híbrida Mamba-Transformer MoE.

Este notebook mostra como usar o Nemotron via NVIDIA NIM API.

## 1. Configurar API Key

A chave está em `~/.bashrc`. Esta célula carrega automaticamente.

In [1]:
import os

def get_nvidia_api_key():
    # Primeiro, verificar se já está no ambiente
    key = os.environ.get("NVIDIA_API_KEY")
    if key:
        return key
    
    # Tentar carregar do bashrc
    bashrc_path = os.path.expanduser("~/.bashrc")
    try:
        with open(bashrc_path, "r") as f:
            for line in f:
                if line.startswith("export NVIDIA_API_KEY="):
                    key = line.split("=", 1)[1].strip().strip('"').strip("'")
                    os.environ["NVIDIA_API_KEY"] = key
                    return key
    except Exception as e:
        print(f"Erro ao ler bashrc: {e}")
    
    return None

NVIDIA_API_KEY = get_nvidia_api_key()

if NVIDIA_API_KEY:
    print(f"✅ NVIDIA_API_KEY carregada: {NVIDIA_API_KEY[:15]}...")
else:
    print("⚠️ NVIDIA_API_KEY não encontrada. Defina em ~/.bashrc ou manualmente:")
    print('os.environ["NVIDIA_API_KEY"] = "sua_chave_aqui"')

✅ NVIDIA_API_KEY carregada: nvapi-8tdO1NjXD...


## 2. Testar conexão com NVIDIA API

In [2]:
import requests

invoke_url = "https://integrate.api.nvidia.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {NVIDIA_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": "nvidia/nemotron-3-nano-30b-a3b",
    "messages": [
        {"role": "user", "content": "Explique o que é Deep Learning em 3 frases."}
    ],
    "temperature": 0.7,
    "max_tokens": 200
}

response = requests.post(invoke_url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print("✅ Resposta do Nemotron:")
    print(result["choices"][0]["message"]["content"])
else:
    print(f"❌ Erro {response.status_code}: {response.text}")

✅ Resposta do Nemotron:

Deep Learning é um ramo do aprendizado de máquina que utiliza redes neurais artificiais com múltiplas camadas ocultas para aprender representações hierárquicas de dados. Essas redes são capazes de extrair características cada vez mais abstratas, desde pixels brutos até objetosComplexos, sem necessidade de engenharia manual das características. Assim, o Deep Learning tem possibilitado avanços significativos em áreas como visão computacional, processamento de linguagem natural e reconhecimento de voz.


## 3. Usando a biblioteca OpenAI (compatível)

In [3]:
# pip install openai
from openai import OpenAI

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY
)

response = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[
        {"role": "user", "content": "Qual a diferença entre RNN e Transformer?"}
    ],
    temperature=0.7,
    max_tokens=300
)

print(response.choices[0].message.content)


## Diferenças principais entre **RNN** (Redes Neurais Recorrentes) e **Transformer**

| Aspecto | RNN (Recorrente) | Transformer |
|---|---|---|
| **Arquitetura básica** | Processa a sequência **um token de cada vez**, mantendo um estado interno (hidden state) que é passado de um passo para o próximo. | Baseado exclusivamente em **mecanismo de atenção** (self‑attention) e em camadas feed‑forward; não há conexão recorrente explícita entre tokens. |
| **Dependência de ordem** | A ordem dos tokens é inerente ao algoritmo: o próximo passo só pode ser calculado depois que o anterior foi processado. | A ordem pode ser informada por **posicionais encodings** adicionados às representações; a atenção pode combinar informações de qualquer posição simultaneamente. |
| **Paral


## 4. Streaming (resposta em tempo real)

In [4]:
print("🤖 Nemotron: ", end="")

for chunk in client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[{"role": "user", "content": "Liste 3 aplicações práticas de LLMs."}],
    temperature=0.7,
    max_tokens=200,
    stream=True
):
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
print()

🤖 Nemotron: 
**3 aplicações práticas de LLMs (Large Language Models)**  

1. **Assistentes virtuais e chatbots corporativos**  
   - Automatizam o atendimento ao cliente, respondendo a dúvidas frequentes, realizando suporte técnico e orientando usuários em processos de compra ou suporte, tudo com linguagem natural e contextualizada.

2. **Geração e resumo de conteúdo**  
   - Produzem textos como artigos, relatórios, legendas para redes sociais, newsletters e roteiros, além de resumir documentos extensos, permitindo que equipes criem material de alta qualidade em menos tempo.

3. **Análise e extração de informações de grandes volumes de texto

IndexError: list index out of range

## 5. Chat com contexto (múltiplas mensagens)

In [5]:
messages = [
    {"role": "system", "content": "Você é um especialista em IA. Seja técnico e conciso."},
    {"role": "user", "content": "O que é atenção em transformers?"},
]

response = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=messages,
    temperature=0.7
)

print(response.choices[0].message.content)


**Atenção em Transformers**  

A atenção (attention) é a operação central que permite que um Transformer “olhe” para todas as posições de uma sequência ao processar cada token. Ela calcula **peso de importância** de cada token de entrada em relação a todos os demais (ou a um conjunto selecionado) antes de gerar a saída.

---

## 1. Atenção Scaled‑Dot‑Product (SDPA)

Para um conjunto de *queries* **Q**, *keys* **K** e *values* **V** (matrizes de dimensão *d_k × n*), a atenção é definida como:

\[
\text{Attention}(Q, K, V) \;=\; \text{softmax}\!\left(\frac{QK^{\top}}{\sqrt{d_k}}\right) V
\]

- **QKᵀ** → produto ponto entre consultas e chaves, gerando uma matriz de compatibilidade de tamanho *n × n*.
- **\sqrt{d_k}** → escala para evitar gradientes explosivos.
- **softmax** → converte as compatibilidades em probabilidades (peso de atenção) na faixa (0,1).
- **\* V** → combina os *values* ponderados pelos pesos de atenção.

---

## 2. Tipos de atenção

| Tipo | Quando usado | Característi

## 6. Parâmetros avançados

In [6]:
response = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b",
    messages=[{"role": "user", "content": "Gere um título criativo para um artigo sobre IA."}],
    temperature=0.9,
    max_tokens=50,
    top_p=0.9
)

print(response.choices[0].message.content)

None


## Modelos Nemotron Disponíveis

| Modelo | Tamanho | Descrição |
|--------|---------|----------|
| `nvidia/nemotron-3-nano-30b-a3b` | 30B (3.5B ativos) | MoE híbrido, reasoning |
| `nvidia/nemotron-4-340b-instruct` | 340B | Modelo grande para tarefas complexas |

### Características:
- **Arquitetura**: Mamba2-Transformer Hybrid MoE
- **Idiomas**: Inglês, Alemão, Espanhol, Francês, Italiano, Japonês
- **Capacidades**: Reasoning, tool calling, chat, código